# Breast Cancer Segmentation TciaBD

We will designing and developing MSGRAP DL model based on H. Lee's research paper. We will adapt it for be trained on breast cancer dataset (CT, MRI, PET) since H. Lee originally trained it on breast cancer ultrasound images. We will note down the differences as we work out the implementation.

[1] H. Lee, J. Park and J. Y. Hwang, "Channel Attention Module With Multiscale Grid Average Pooling for Breast Cancer Segmentation in an Ultrasound Image," in IEEE Transactions on Ultrasonics, Ferroelectrics, and Frequency Control, vol. 67, no. 7, pp. 1344-1353, July 2020, doi: [10.1109/TUFFC.2020.2972573](https://ieeexplore-ieee-org.libaccess.sjlibrary.org/document/8988165)

Referenced 2D Channel - Spatial Attention (Squeeze & Excite) VGG based TensorFlow Keras code in tutorial [Attending to Channels Using Keras and TensorFlow](https://pyimagesearch.com/2022/05/30/attending-to-channels-using-keras-and-tensorflow/)

## Outline

- Prepare Tcia Breast Diagnosis Data
- Breast Segmentation Model Architecture
- Train Breast Segmentation ML/DL Models
- Evaluate Breast Segmentation ML/DL Models Quantitatively
- Evaluate Breast Segmentation ML/DL Models Qualitatively
- Deploy Breast Segmentation DL Model for Inference

In [1]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate,Dropout
from tensorflow.keras.layers import Multiply, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-19 20:46:35.927064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 20:46:36.487513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2022-11-19 20:46:36.487563: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-19 20:46:36.566579: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-19 20:46:37.773

## Prepare Tcia Breast Diagnosis Data

## Breast Segmentation Model Architecture

DL MSGRAP Breast Cancer Segmentation Architecture:

- Encoder
    - Based on VGGNet except for batch normalization and channel attention modules
    - Use all conv layers with C' filters with size 3x3xC
    - Upsample the feature maps using a 4x4 transpose conv with a stride of 2
    - Unlike UNet, only 2 feature maps from encoder are connected to decoder
- Decoder
    - symmetrical encoder architecture built into decoder
    - Use all conv layers with C' filters with size 3x3xC
    - Final conv layer has 2 filters with size 3x3x64
    - Obtain final segmentation results into binary classes via argmax with threshold 0.5

The network receives a breast ultrasound image as input and predicts its semantic segmentation result.

- Note: C and C' are the previous and current number of feature maps, except for the final conv layer

- Note: batch normalization is highly influenced by a batch size: the smaller the batch size, the lower the performance is. Small batch size reduces the generalization ability. H. Lee et al uses group batch normalization since it has little effect on batch size and the dataset used in experiments had enough spatial size

- Note: group normalization divides each channel into N groups and normalizes the features within each group regardless of the batch size. Thus, it doesnt depend on the batch size and can overcome the generalization issues caused by the small batch size when the network is trained with large input imagess.

- Note: After performing additional experiment, H. Lee et al demonstrated that their  network architecture with 2 feature maps connected between the encoder and decoder performed better than UNet like architectures.
    - It is better to not use low-level features in the network since most ultrasound images are noisy. (similar for PET, SPECT).
    
- Note: H. Lee et al also conducts the ablation study for architecture with two of the tenfold datasets.

- Note: Semantic segmentation F1 Score: UNet = 0.78, H. Lee's MSGRAP = 0.79

![msgrap_h_lee_breast_cancer_segmentation](./msgrap_h_lee_breast_cancer_segmentation.jpg)


In [2]:
class model():
    def __init__(self, input_shape, ratio, blocks, num_classes,
        dense_units, conv_filters, is_squeeze_excite, optimizer,
        loss, metrics):
        self.input_shape = input_shape
        self.rate = ratio
        self.block = blocks
        self.num_classes = num_classes
        
        self.dense_units = dense_units
        self.conv_filters = conv_filters
        
        # Flag decides whether to add squeeze excitation layer
        # to network. This is channel - spatial attention layer
        self.is_squeeze_excite = is_squeeze_excite
        
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        pass

    def split(self, resized_img, resized_mask):
        pass
    
    def data_gen(self, img_list, mask_list, batch_size):
        pass
    
#     def conv_stem(self, inputs, filters=3, kernel_size=3):
#         # pass the input through a CONV => ReLU layer block
#         x = Conv3D(filters = filters, kernel_size = (kernel_size, kernel_size, kernel_size),
#             kernel_initializer = "he_normal", padding = "same")(inputs)
#         x = Activation("relu")(x)
#         x = GroupNormalization()(x)
#         if self.is_squeeze_excite:
#             x = self.squeeze_excite_block(x)
#             x = Activation('relu')(x)
#         return x
    
#     def learner(self, x):
#         # build the learner by stacking 3D convolutional layer blocks
#         for num_layers, num_filters in self.blocks:
#             x = self.convolutional_block(x, num_layers, num_filters)
#         return x
    
    def convolutional_block(self, x, filters=3, kernel_size=3, num_blocks=2):
        """conv layer followed by group normalization"""
        """ iterate over the number of layers and build a block
            with 3D convolutional layers"""
        for i in range(num_blocks):
            x = Conv3D(filters = filters, kernel_size = (kernel_size, kernel_size, kernel_size),
                       kernel_initializer = 'he_normal', padding = 'same')(x)
            x = Activation('relu')(x)
            x = GroupNormalization()(x)
            if self.is_squeeze_excite:
                x = self.squeeze_excite_block(x)
                x = Activation('relu')(x)
        return x
    
    def squeeze_excite_block(self, x):
        # store the input
        shortcut = x
        
        # calculate the number of filters the input has
        filters = x.shape[-1]
        
        # the squeeze operation reduces the input dimensionality
        # here we do a global average pooling across the filters,
        # which reduces the input to a 2D feature map
        x = GlobalAveragePooling3D(keepdims=True)(x)
    
        # reduce the number of filters to C/r
        x = Dense(filters // self.ratio, activation="relu",
            kernel_initializer="he_normal", use_bias=False)(x)
    
        # the excitation operation restores the input dimensionality
        x = Dense(filters, activation="sigmoid",
            kernel_initializer="he_normal", use_bias=False)(x)
        
        # multiply the attention weights with the original input
        x = Multiply()([shortcut, x])
    
        # return the output of the SE block
        return x
    
    def msgrap(input_img, filters=64, dropout=0.2):
        """MSGRAP based on 3D UNet and Squeeze Excitation Network
           Squeeze Excitation is the channel - spatial attention layer(s)
           This implementation is based on the diagram above. It initially
           is initially designed for breast cancer ultrasound, so we will
           see how it does for CT, MRI, PET.
        """
        # 64 filters for first two 3D conv blocks
        conv1 = convolutional_block(input_img, filters*1, kernel_size=3,
                    num_blocks=2)
        
        # MaxPooling3D, then 128 filters for next two 3D conv blocks
        pool2 = MaxPooling3D((2,2,2))(conv1)
        conv2 = convolutional_block(pool2, filters*2, kernel_size=3,
                    num_blocks=2)
        
        # MaxPooling3D, then 256 filters for next three 3D conv blocks
        pool3 = MaxPooling3D((2,2,2))(conv2)
        conv3 = convolutional_block(pool3, filters*4, kernel_size=3,
                    num_blocks=3)
        
        # MaxPooling3D, then 512 filters for next three 3D conv blocks
        pool4 = MaxPooling3D((2,2,2))(conv3)
        conv4 = convolutional_block(pool4, filters*8, kernel_size=3,
                    num_blocks=3)
        
        # MaxPooling3D, then 512 filters for next three 3D conv blocks
        pool5 = MaxPooling3D((2,2,2))(conv4)
        conv5 = convolutional_block(pool5, filters*8, kernel_size=3,
                    num_blocks=3)
        
        # Conv3DTranspose, concatenation w/ conv4, then 512 filters for
        # next three 3D conv blocks
        ups6 = Conv3DTranspose(filters*8, (4,4,4), strides=(2,2,2), padding="same",
                              activation="relu", kernel_initializer="he_normal")(conv5)
        ups6 = concatenate([ups6, conv4])
        conv6 = convolutional_block(ups6, filters*8, kernel_size=3,
                    num_blocks=3)
        
        # Conv3DTranspose, concatenation w/ conv3, then 256 filters for
        # next three 3D conv blocks
        ups7 = Conv3DTranspose(filters*4, (4,4,4), strides=(2,2,2), padding="same",
                              activation="relu", kernel_initializer="he_normal")(conv6)
        ups7 = concatentate([ups7, conv3])
        conv7 = convolutional_block(ups7, filters*4, kernel_size=3,
                    num_blocks=3)
        
        # Conv3DTranpose, then 128 filters for next two 3D conv blocks
        ups8 = Conv3DTranpose(filters*2, (4,4,4), strides=(2,2,2), padding="same",
                             activation="relu", kernel_initializer="he_normal")(conv7)
        conv8 = convolutional_block(ups8, filters*2, kernel_size=3,
                    num_blocks=2)
        
        # Conv3DTranspose, then 64 filters for next two 3D conv blocks
        ups9 = Conv3DTranspose(filters*1, (4,4,4), strides=(2,2,2), padding="same",
                              activation="relu", kernel_initializer="he_normal")(conv8)
        conv9 = convolutional_block(ups9, filters*1, kernel_size=3,
                    num_blocks=2)
        # Final conv layer has 2 filters with size 3x3x64
        # Obtain final segmentation results into binary classes via argmax with threshold 0.5
        outputs = Conv3D(2, (3,3,64), activation="sigmoid", padding="same")(conv9)
        model = Model(inputs=[input_img], outputs=[outputs])
        return model

## Train Breast Segmentation ML/DL Models

To evaluate the performance of their proposed networks (MSGRAP, etc), H. Lee et al trained different models, such as 

- FCN
- UNet
- SegNet
- PSPNet-18

Then compared their performance.

Loss function:

`L_theta_k_D = -(M-1)_sum_(c=0) (GT_c)log(f(I_theta)_c)`

- Note: GT_c is the predicted probability and the binary indicator for the class, c
- Note: breast cancer segmentation is a binary classification for each pixel, use `M = 2`

- Note: since the breast ultrasound cancer datasets were limited, H. Lee et al  configured the training and testing processes as tenfold cross-validation.

- Note: Divided the data into 146 or 147 breast cancer ultrasound images for training
    - 16 or 17 breast cancer ultrasound images for testing in each validation step

- Note: H. Lee et al agumented each patch by random horizontal and vertical flips and random 90 deg rotations

- Note: for training and testing, all image sizes were set to average 454x537 pixels

- Note: for training, the weights of all conv layers were initialized `Kaiming initialization`

Note: Adam optimization method with parameters was used:

- Beta_1 = 0.9
- Beta_2 = 0.999
- epsilon = 10^-8

- Note: `Learning rate = 10^-3` and was `reduced by half` every `30 epochs`

- Note: `mini-batch size = 8`

- Note: models were trained for `120 epochs`

- Note: PyTorch was used to implement and train networks

NOTE: it took four days to train the models using:

- Intel Zeon E5-2620 at 2.0 GHz
- NVIDIA TITAN RTX (24GB)

## Evaluate Breast Segmentation ML/DL Models Quantitatively

### Performance Metrics

For quantitative comparisions of DL MSGRAP with other methods, H. Lee et al used global accuracy, F1 score, sensitivity, specificity.

Accuracy = `(TP+TN)/(TP+FP+FN+TN)`

F1 = `(2*TP)/(2*TP+FP+FN)`

IoU = `TP/(TP+FP+FN)`

- Note: Accuracy is most basic metric for several CV tasks
- Note: F1 score is good for imbalanced data. For ex, in H. Lee et al's dataset, there was a small portion of cancer among all breast ultrasound images, this data set can be considered imbalanced.
- Note: this dataset was imbalanced, it consists of 5% of cancer pixels and 95% of nomal pixels. So, H. Lee et al used FPR, precision, intersection over union (IoU), and area under the curve (AUC) of precision and recall (PR) for fair evaluation.
- Note: AUC metrics used a sweep of the threshold from `p=0` to `p=1` as opposed to `p=0.5` (argmax) used for the remaining non-AUC metrics.
- Note: FPR is number of false positive over the one of the condition negatives.
- Note: IoU metric is commonly used in semantic segmentation

Show metrics in a table for models:

- FCN, UNet, SegNet, PSPNet-18, ENCNet-18
- Ours-GAP, Ours-GRAP, Ours-MSGRAP

H. Lee et al didn't use semantic segmentation networks based on ResNet-34, 51 and DesneNet that have more than 18 conv layers, so there would be fair comparisons since those networks have many more parameters than theirs.

- Ours-MSGRAP had better `F1 score = 0.7658` than other models
    - FCN = 0.7123, UNet = 0.7132, SegNet = 0.7225, PSPNet-18 = 0.7520, ENCNet-18 = 0.7266
    
Ours-MSGRAP showed higher performance than other models in global accuracy, specificity, FPR, precision and IoU